In [1]:
import cv2
import torch
import llama
import clipvpr
from PIL import Image

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model, process = clipvpr.load(clip_name="ViT-B/16", device = device, llama_name="BIAS-7B", llama_dir='./path/to/LLaMA/', llama_type="7B", 
         llama_download_root='ckpts', max_seq_len=512, phase="finetune", 
         prompt=['Is the scene in the picture urban or rural? How many lanes are there on the road in the photo? Is there a residential building in the picture? If so, which side of the road is it located on? Are there vegetation and trees in the photo? If so, which side of the road is it located on?'])

In [3]:
model.eval().to(device)

CLIPVPR(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
       

In [4]:
prompts_list = 2 * ['Is there a sidewalk on the road in the picture?']

prompts = [llama.format_prompt(prompt) for prompt in prompts_list]
img1 = Image.fromarray(cv2.imread("./data/image1.jpg"))
img2 = Image.fromarray(cv2.imread("./data/image2.jpg"))
imgs = [img1, img2]
img_load = torch.stack([process(img).to(device) for img in imgs], dim=0)
print(img_load.shape)

torch.Size([2, 3, 224, 224])


In [5]:
result = model(img_load)

2


RuntimeError: Input 
['The scene in the picture is rural. There are two lanes on the road in the photo. There is a residential building in the picture. It is located on the right side of the road. There are vegetation and trees in the photo. It is located on the right side of the road.']

### Instruction:
['Is the scene in the picture urban or rural? How many lanes are there on the road in the photo? Is there a residential building in the picture? If so, which side of the road is it located on? Are there vegetation and trees in the photo? If so, which side of the road is it located on?']

### Response:
['The scene in the picture is rural. There are two lanes on the road in the photo. There is a residential building in the picture. It is located on the right side of the road. There are vegetation and trees in the photo. It is located on the right side of the road.']

### Instruction:
['Is the scene in the picture urban or rural? How many lanes are there on the road in the photo? Is ther